## 8th January

understanding the tokenizer in GPT-1

trying to implement the tokenizer in GPT-1

In [17]:
import collections
from collections import defaultdict
pairs = defaultdict(int)

In [18]:
vocab = {"hello": 5, "world": 3}
new_vocab = {}

In [19]:
for token in vocab.keys():
    ntoken = ' '.join(list(token)) + ' </w>'
    new_vocab[ntoken] = vocab[token]
    print(ntoken)
print(new_vocab)

h e l l o </w>
w o r l d </w>
{'h e l l o </w>': 5, 'w o r l d </w>': 3}


In [20]:
for word, freq in new_vocab.items():
    symbols = word.split()
    for i in range(len(symbols)-1):
        pairs[symbols[i], symbols[i+1]] += freq

In [21]:
pairs

defaultdict(int,
            {('h', 'e'): 5,
             ('e', 'l'): 5,
             ('l', 'l'): 5,
             ('l', 'o'): 5,
             ('o', '</w>'): 5,
             ('w', 'o'): 3,
             ('o', 'r'): 3,
             ('r', 'l'): 3,
             ('l', 'd'): 3,
             ('d', '</w>'): 3})

In [22]:
max(pairs, key=pairs.get)

('h', 'e')

### 9th January

Working on building Byte Pair Encoding tokenizer

In [47]:
from typing import List, Dict
from tqdm import tqdm
import nltk, os
from nltk import sent_tokenize, wordpunct_tokenize
from collections import defaultdict


class BPETokenizer:
    def __init__(self, ):
        pass
    
    @staticmethod
    def train_tokenizer(file_paths: List[str], min_freq: int, merges: int) -> 'BPETokenizer':
        vocab = create_vocab(file_paths)
        truncate_vocab(vocab, min_freq)
        bpe_vocab = create_bpe_vocab(vocab)


def create_vocab(file_paths:  List[str]) -> Dict[str, int]:
    vocab = defaultdict(int)
    for file_path in tqdm(file_paths, desc='Reading files'):
        text = open(file_path, 'r', encoding = 'utf-8-sig').read()
        sentences = sent_tokenize(text)
        for sentence in sentences:
            tokens = wordpunct_tokenize(sentence)
            for token in tokens:
                vocab[token] +=1
    return vocab

def truncate_vocab(vocab: Dict[str, int], min_freq: int) -> None:
    tokens = list(vocab.keys()) 
    for token in tokens:
        if vocab[token] < min_freq:
            del vocab[token]
def create_bpe_vocab(vocab: Dict[str, int]):
    bpe_vocab = {}
    for token in vocab:
        new_token = ' '.join(list(token)) + ' </w>'
        bpe_vocab[new_token] = vocab[token]
    return bpe_vocab




In [40]:
from typing import List, Dict
from tqdm import tqdm
import nltk, os
from nltk import sent_tokenize, wordpunct_tokenize
from collections import defaultdict

def create_vocab(file_paths:  List[str]) -> Dict[str, int]:
    vocab = defaultdict(int)
    for file_path in tqdm(file_paths, desc='Reading files'):
        text = open(file_path, 'r', encoding = 'utf-8-sig').read()
        sentences = sent_tokenize(text)
        for sentence in sentences:
            tokens = wordpunct_tokenize(sentence)
            for token in tokens:
                vocab[token] +=1
    return vocab

file_paths = os.listdir('Data')
file_paths = [os.path.join('Data', file_path) for file_path in file_paths]
print(file_paths)
breaking_bad_vocab = create_vocab(file_paths)

['Data\\BreakingBadSeason1.txt', 'Data\\BreakingBadSeason2.txt', 'Data\\BreakingBadSeason3.txt', 'Data\\BreakingBadSeason4.txt', 'Data\\BreakingBadSeason5.txt']


Reading files: 100%|██████████| 5/5 [00:01<00:00,  3.30it/s]


In [46]:
print(len(breaking_bad_vocab))
print(sorted(breaking_bad_vocab.items(), key=lambda x:x[1], reverse=True)[:50])

6245
[('.', 4685), (',', 3930), ("'", 2722), (":'''", 2573), ("''':", 2378), ('I', 2213), ('you', 1815), ('?', 1710), ('the', 1507), ('to', 1406), ('Walter', 1308), ('s', 1094), ('a', 1046), ('and', 861), ('Jesse', 843), ('it', 810), ('that', 772), ('of', 714), ('t', 671), ("]''", 641), ('You', 582), ('is', 577), ('me', 567), ('!', 562), ('in', 553), ('...', 543), (':', 500), ("''", 456), ('this', 435), ("''[", 434), ('know', 388), ('w', 385), ('your', 382), ('what', 379), ('for', 370), ('my', 363), ('-', 359), ('re', 358), ('he', 355), ('do', 349), ('on', 347), ('just', 325), ('m', 324), ('Hank', 312), ('not', 311), ('we', 308), ('Skyler', 307), ('Saul', 288), ('What', 285), ('<', 281)]


In [50]:
def truncate_vocab(vocab: Dict[str, int], min_freq: int) -> None:
    tokens = list(vocab.keys()) 
    for token in tokens:
        if vocab[token] < min_freq:
            del vocab[token]

In [51]:
truncate_vocab(vocab=breaking_bad_vocab, min_freq= 300)

In [54]:
print(sorted(breaking_bad_vocab.items(), key=lambda x:x[1])[:50])

[('Skyler', 307), ('we', 308), ('not', 311), ('Hank', 312), ('m', 324), ('just', 325), ('on', 347), ('do', 349), ('he', 355), ('re', 358), ('-', 359), ('my', 363), ('for', 370), ('what', 379), ('your', 382), ('w', 385), ('know', 388), ("''[", 434), ('this', 435), ("''", 456), (':', 500), ('...', 543), ('in', 553), ('!', 562), ('me', 567), ('is', 577), ('You', 582), ("]''", 641), ('t', 671), ('of', 714), ('that', 772), ('it', 810), ('Jesse', 843), ('and', 861), ('a', 1046), ('s', 1094), ('Walter', 1308), ('to', 1406), ('the', 1507), ('?', 1710), ('you', 1815), ('I', 2213), ("''':", 2378), (":'''", 2573), ("'", 2722), (',', 3930), ('.', 4685)]


In [55]:
def create_bpe_vocab(vocab: Dict[str, int]):
    bpe_vocab = {}
    for token in vocab:
        new_token = ' '.join(list(token)) + ' </w>'
        bpe_vocab[new_token] = vocab[token]
    return bpe_vocab

In [58]:
bpe_bb_vocab = create_bpe_vocab(breaking_bad_vocab)
print(sorted(bpe_bb_vocab.items(), key=lambda x:x[1])[:50])

[('S k y l e r </w>', 307), ('w e </w>', 308), ('n o t </w>', 311), ('H a n k </w>', 312), ('m </w>', 324), ('j u s t </w>', 325), ('o n </w>', 347), ('d o </w>', 349), ('h e </w>', 355), ('r e </w>', 358), ('- </w>', 359), ('m y </w>', 363), ('f o r </w>', 370), ('w h a t </w>', 379), ('y o u r </w>', 382), ('w </w>', 385), ('k n o w </w>', 388), ("' ' [ </w>", 434), ('t h i s </w>', 435), ("' ' </w>", 456), (': </w>', 500), ('. . . </w>', 543), ('i n </w>', 553), ('! </w>', 562), ('m e </w>', 567), ('i s </w>', 577), ('Y o u </w>', 582), ("] ' ' </w>", 641), ('t </w>', 671), ('o f </w>', 714), ('t h a t </w>', 772), ('i t </w>', 810), ('J e s s e </w>', 843), ('a n d </w>', 861), ('a </w>', 1046), ('s </w>', 1094), ('W a l t e r </w>', 1308), ('t o </w>', 1406), ('t h e </w>', 1507), ('? </w>', 1710), ('y o u </w>', 1815), ('I </w>', 2213), ("' ' ' : </w>", 2378), (": ' ' ' </w>", 2573), ("' </w>", 2722), (', </w>', 3930), ('. </w>', 4685)]


In [ ]:
from tqdm import trange
for i in trange(10):
    print(i)

100%|██████████| 10/10 [00:00<00:00, 10007.88it/s]

0
1
2
3
4
5
6
7
8
9


In [66]:
from typing import Tuple

def get_stats(vocab: Dict[str, int]) -> Dict[Tuple[str, str], int]:
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs


In [67]:
print(get_stats(bpe_bb_vocab))

defaultdict(<class 'int'>, {(':', "'"): 2573, ("'", "'"): 11433, ("'", '</w>'): 6392, ('w', '</w>'): 773, (':', '</w>'): 2878, ('a', '</w>'): 1046, ('-', '</w>'): 359, (',', '</w>'): 3930, ('W', 'a'): 1308, ('a', 'l'): 1308, ('l', 't'): 1308, ('t', 'e'): 1308, ('e', 'r'): 1615, ('r', '</w>'): 2367, ('t', 'h'): 2714, ('h', 'a'): 1151, ('a', 't'): 1151, ('t', '</w>'): 3268, ('h', 'e'): 1862, ('e', '</w>'): 3938, ('.', '</w>'): 5228, ('t', 'o'): 1406, ('o', '</w>'): 1755, ('o', 'f'): 714, ('f', '</w>'): 714, ('a', 'n'): 1173, ('n', 'd'): 861, ('d', '</w>'): 861, ('f', 'o'): 370, ('o', 'r'): 370, ('i', 's'): 1012, ('s', '</w>'): 2106, ('I', '</w>'): 2213, ('h', 'i'): 435, ('n', 'o'): 699, ('o', 't'): 311, ('m', 'y'): 363, ('y', '</w>'): 363, ('S', 'k'): 307, ('k', 'y'): 307, ('y', 'l'): 307, ('l', 'e'): 307, ('y', 'o'): 2197, ('o', 'u'): 2779, ('u', '</w>'): 2397, ('k', 'n'): 388, ('o', 'w'): 388, ('r', 'e'): 358, ('.', '.'): 1086, ('m', 'e'): 567, ('i', 'n'): 553, ('n', '</w>'): 900, ('j'

In [69]:
pairs = get_stats(bpe_bb_vocab)
best = max(pairs, key=pairs.get)
print(best)

("'", "'")


In [72]:
import re
def merge_vocab(pair: Tuple[str, str], vocab: Dict[str, int]) -> Dict[str,int]:
    merged_vocab = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in vocab:
        new_word = p.sub(''.join(pair), word)
        merged_vocab[new_word] = vocab[word]
    return merged_vocab


In [75]:
bpe_bb_vocab

{": ' ' ' </w>": 2573,
 'w </w>': 385,
 ': </w>': 500,
 'a </w>': 1046,
 '- </w>': 359,
 ', </w>': 3930,
 'W a l t e r </w>': 1308,
 't h a t </w>': 772,
 'h e </w>': 355,
 '. </w>': 4685,
 't o </w>': 1406,
 'o f </w>': 714,
 't h e </w>': 1507,
 'a n d </w>': 861,
 'f o r </w>': 370,
 'i s </w>': 577,
 "' </w>": 2722,
 's </w>': 1094,
 'I </w>': 2213,
 't h i s </w>': 435,
 'n o t </w>': 311,
 'm y </w>': 363,
 'S k y l e r </w>': 307,
 'y o u </w>': 1815,
 'k n o w </w>': 388,
 'r e </w>': 358,
 '. . . </w>': 543,
 'm e </w>': 567,
 'i n </w>': 553,
 'j u s t </w>': 325,
 'i t </w>': 810,
 'J e s s e </w>': 843,
 '? </w>': 1710,
 "' ' ' : </w>": 2378,
 't </w>': 671,
 'w h a t </w>': 379,
 'o n </w>': 347,
 'Y o u </w>': 582,
 'y o u r </w>': 382,
 'm </w>': 324,
 '! </w>': 562,
 "] ' ' </w>": 641,
 "' ' </w>": 456,
 "' ' [ </w>": 434,
 'd o </w>': 349,
 'w e </w>': 308,
 'H a n k </w>': 312}

In [83]:
merged_bb_vocab = merge_vocab(('W', 'a'), bpe_bb_vocab)
print(merged_bb_vocab)

{": ' ' ' </w>": 2573, 'w </w>': 385, ': </w>': 500, 'a </w>': 1046, '- </w>': 359, ', </w>': 3930, 'Wa l t e r </w>': 1308, 't h a t </w>': 772, 'h e </w>': 355, '. </w>': 4685, 't o </w>': 1406, 'o f </w>': 714, 't h e </w>': 1507, 'a n d </w>': 861, 'f o r </w>': 370, 'i s </w>': 577, "' </w>": 2722, 's </w>': 1094, 'I </w>': 2213, 't h i s </w>': 435, 'n o t </w>': 311, 'm y </w>': 363, 'S k y l e r </w>': 307, 'y o u </w>': 1815, 'k n o w </w>': 388, 'r e </w>': 358, '. . . </w>': 543, 'm e </w>': 567, 'i n </w>': 553, 'j u s t </w>': 325, 'i t </w>': 810, 'J e s s e </w>': 843, '? </w>': 1710, "' ' ' : </w>": 2378, 't </w>': 671, 'w h a t </w>': 379, 'o n </w>': 347, 'Y o u </w>': 582, 'y o u r </w>': 382, 'm </w>': 324, '! </w>': 562, "] ' ' </w>": 641, "' ' </w>": 456, "' ' [ </w>": 434, 'd o </w>': 349, 'w e </w>': 308, 'H a n k </w>': 312}


In [87]:
def create_bytes_freq(vocab: Dict[str, int]) -> Dict[str, int]:
    byte_freq = defaultdict(int)
    for word, freq in vocab.items():
        bytes_ = word.split(' ')
        for byte in bytes_:
            byte_freq[byte] += 1
    
    for token in ['<line/>', '</line>', '<pad>', '<unk>']:
        byte_freq[token] += 1
    return byte_freq


In [88]:
bytes_vocab = create_bytes_freq(merged_bb_vocab)
print(bytes_vocab)

defaultdict(<class 'int'>, {':': 3, "'": 13, '</w>': 47, 'w': 4, 'a': 5, '-': 1, ',': 1, 'Wa': 1, 'l': 2, 't': 11, 'e': 9, 'r': 5, 'h': 5, '.': 4, 'o': 10, 'f': 2, 'n': 6, 'd': 2, 'i': 4, 's': 6, 'I': 1, 'm': 3, 'y': 4, 'S': 1, 'k': 3, 'u': 4, 'j': 1, 'J': 1, '?': 1, 'Y': 1, '!': 1, ']': 1, '[': 1, 'H': 1, '<line/>': 1, '</line>': 1, '<pad>': 1, '<unk>': 1})


In [91]:
def create_vocab_maps(vocab: Dict[str, int]) -> (Dict[str, int], Dict[int, str]):
    ordered_freqs = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
    vocab_to_idx, idx_to_vocab = {}, {}
    for i in range(len(ordered_freqs)):
        word, freq = ordered_freqs[i]
        vocab_to_idx[word] = i
        idx_to_vocab[i] = word
    return vocab_to_idx, idx_to_vocab


In [93]:
vocab_to_idx, idx_to_vocab = create_vocab_maps(bytes_vocab) 
print(vocab_to_idx, idx_to_vocab)

{'</w>': 0, "'": 1, 't': 2, 'o': 3, 'e': 4, 'n': 5, 's': 6, 'a': 7, 'r': 8, 'h': 9, 'w': 10, '.': 11, 'i': 12, 'y': 13, 'u': 14, ':': 15, 'm': 16, 'k': 17, 'l': 18, 'f': 19, 'd': 20, '-': 21, ',': 22, 'Wa': 23, 'I': 24, 'S': 25, 'j': 26, 'J': 27, '?': 28, 'Y': 29, '!': 30, ']': 31, '[': 32, 'H': 33, '<line/>': 34, '</line>': 35, '<pad>': 36, '<unk>': 37} {0: '</w>', 1: "'", 2: 't', 3: 'o', 4: 'e', 5: 'n', 6: 's', 7: 'a', 8: 'r', 9: 'h', 10: 'w', 11: '.', 12: 'i', 13: 'y', 14: 'u', 15: ':', 16: 'm', 17: 'k', 18: 'l', 19: 'f', 20: 'd', 21: '-', 22: ',', 23: 'Wa', 24: 'I', 25: 'S', 26: 'j', 27: 'J', 28: '?', 29: 'Y', 30: '!', 31: ']', 32: '[', 33: 'H', 34: '<line/>', 35: '</line>', 36: '<pad>', 37: '<unk>'}


### 10th January

Didn't do anything related to Tech stuff

### 11th January

Tried to understand the tokenize_dataset in GPT1 and understood few methods like merge_byte and other supporting methods in it.

## 12th January

understanding the code related to tokenize_dataset

Didn't get the time do a bit of coding as I was fully occupied with Kondal's discussion in the evening about life

### 13th January

started of the day with RNN's vanishing gradients problem.(because of the repeated multiflication of the same weight matrix in backward propagation as it was recurrent with same weight with different time steps(we calculate loss for the timesteps))

how it was handled in the LSTM

### 14th January

Going through the interview and other process for RecVue